In [1]:
import networkx as nx
from cvrp_ortools_base import solve_cvrp
from collections import Counter


In [2]:
# Получить матрицу времени перемещений из графа для определенных узлов
def get_time_matrix(G_init):
    G_demand = G_init.copy()
    # Reindex nodes
    i = 0
    index_mapping = {}
    for node in G_demand.nodes:
        index_mapping[node] = i
        i += 1
    G = nx.relabel_nodes(G_demand, index_mapping, copy=False)
    max_index = i

    # Convert to time matrix
    time_matrix = []
    for i in range(max_index):
        l = []
        for j in range(max_index):
            if i == j:
                l.append(0)
            else:
                l.append(round(G[i][j]["weight"], 2))
        time_matrix.append(l)

    return time_matrix, index_mapping


In [3]:
# Получить массив спроса по узлам за период
def calc_demand(G_init, period: list, index_mapping):
    demand_dicts = []
    for date in period:
        demand_dict = nx.get_node_attributes(G_init, date)
        demand_dicts.append(demand_dict)
    # Суммировать спрос за период по каждому узлу
    accum_demand = Counter(demand_dicts[0])
    for i in range(1, len(demand_dicts)):
        accum_demand = accum_demand + Counter(demand_dicts[i])
    # Для использования в or-tools спрос должен быть представлен в виде массива, где узел идентифицируется индексом
    demand_list = list(accum_demand.values())
    # Включить узел депо с нулевым спросом
    start_node_index = index_mapping[start_node]
    demand_list.insert(start_node_index, 0)
    accum_demand[start_node] = 0
    return demand_list, accum_demand

In [4]:
# Граф Василеостровского района со значениями спроса по суткам
G_demand = nx.read_graphml('vrp_project/datasets/graph_with_demand.graphml')
dates = ['2023-03-01', '2023-03-02', '2023-03-03', '2023-03-04', '2023-03-05']
# Даты, за которые рассматривается спрос - считаем, что это прогнозные данные спроса
# dates = ['2023-03-01']
# Ограничение количества узлов с потреблением для моделирования проблемы
# ЗДЕСЬ МОГУТ БЫТЬ ПЕРЕЧИСЛЕНЫ ТОЛЬКО УЗЛЫ СО СПРОСОМ
chosen_points = ['113', '47', '100', '87', '200', '16', '46', '63', '69', '83']
# Назначение узла-депо
start_node = '15'
# Задать доступное количество транспортных средств (фиксированное)
num_vehicles = 4
# Из графа исключаются все узлы, кроме выбранных узлов с потреблением и узла-депо
G_demand.remove_nodes_from(list(n for n in G_demand.nodes if n not in chosen_points and n != start_node))

# Получить матрицу времени перемещений для "обрезанного" графа
time_matrix, mapping = get_time_matrix(G_demand)
time_matrix = [[int(x) for x in row] for row in time_matrix]
# Получить индекс узла депо
depo_index = mapping[start_node]
# Для каждой даты решить задачу CVRP
total_time = 0
vehicles_by_date = {}
for date in dates:
    print(date)
    demand, demand_dict = calc_demand(G_demand, [date], mapping)
    print('Cпрос на дату: ', dict(demand_dict))
    print('')
    # Запустить решение CVRP при заданных параметрах
    vehicles_trips, covered_nodes, routing_time = solve_cvrp(time_matrix, demand, depo_index, num_vehicles, mapping)
    # Определить, сколько машин было задействовано
    vehicles = 0
    for k in vehicles_trips.keys():
        if vehicles_trips[k]>0:
            vehicles+=1
    vehicles_by_date[date] = vehicles
    # Накопить общее время машин в пути
    total_time += routing_time
    print('---------------------------------------------------------------------')
    print('')

print('Выезжало машин:')
print(vehicles_by_date)
print(f'Общее время в пути: {round(total_time/60,2)} час.')

2023-03-01
Cпрос на дату:  {'15': 0, '16': 229, '46': 35, '47': 528, '63': 743, '69': 331, '83': 445, '87': 456, '113': 457, '200': 399}

Route for vehicle 0:
Node '15' Delivered(0) ->  Node '15' Delivered(0)
Time of the route: 0min
Load of the route: 0

Route for vehicle 1:
Node '15' Delivered(0) ->  Node '15' Delivered(0)
Time of the route: 0min
Load of the route: 0

Route for vehicle 2:
Node '15' Delivered(0) -> Node '69' Delivered(743) -> Node '63' Delivered(1271) -> Node '100' Delivered(1727) -> Node '46' Delivered(1956) -> Node '47' Delivered(1991) ->  Node '15' Delivered(1991)
Time of the route: 148min
Load of the route: 1991

Route for vehicle 3:
Node '15' Delivered(0) -> Node '83' Delivered(331) -> Node '87' Delivered(776) -> Node '113' Delivered(1233) -> Node '200' Delivered(1632) -> Node '16' Delivered(1779) ->  Node '15' Delivered(1779)
Time of the route: 196min
Load of the route: 1779

Total time of all routes: 344min
Total load of all routes: 3770
------------------------

## GNN

In [18]:
import torch
import os
import numpy as np
from torch_geometric.data import Data, DataLoader
from VRP.creat_vrp import reward1,creat_instance
from VRP.VRP_Actor import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

In [19]:
def discrete_cmap(N, base_cmap=None):
    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

def plot_vehicle_routes(data, route, ax1, markersize=5, visualize_demands=False, demand_scale=1, round_demand=False):

    plt.rc('font', family='Times New Roman', size=10)

    routes = [r[r != 0] for r in np.split(route.cpu().numpy(), np.where(route.cpu().numpy() == 0)[0]) if (r != 0).any()]
    print(routes)
    depot = data.x[0].cpu().numpy()
    locs = data.x[1:].cpu().numpy()
    demands = data.demand.cpu().numpy()
    demands=demands[1:]

    capacity = data.capcity

    x_dep, y_dep = depot
    ax1.plot(x_dep, y_dep, 'sk', markersize=markersize * 4)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)

    legend = ax1.legend(loc='upper center')

    cmap = discrete_cmap(len(routes) + 2, 'nipy_spectral')
    dem_rects = []
    used_rects = []
    cap_rects = []
    qvs = []
    total_dist = 0
    for veh_number, r in enumerate(routes):
        color = cmap(len(routes) - veh_number)  # Invert to have in rainbow order

        route_demands = demands[r - 1]
        coords = locs[r - 1, :]
        xs, ys = coords.transpose()

        total_route_demand = sum(route_demands)
        #assert total_route_demand <= capacity
        if not visualize_demands:
            ax1.plot(xs, ys, 'o', mfc=color, markersize=markersize, markeredgewidth=0.0)

        dist = 0
        x_prev, y_prev = x_dep, y_dep
        cum_demand = 0
        for (x, y), d in zip(coords, route_demands):
            dist += np.sqrt((x - x_prev) ** 2 + (y - y_prev) ** 2)
            cap_rects.append(Rectangle((x, y), 0.01, 0.1))
            used_rects.append(Rectangle((x, y), 0.01, 0.1 * total_route_demand / capacity))
            dem_rects.append(Rectangle((x, y + 0.1 * cum_demand / capacity), 0.01, 0.1 * d / capacity))

            x_prev, y_prev = x, y
            cum_demand += d

        dist += np.sqrt((x_dep - x_prev) ** 2 + (y_dep - y_prev) ** 2)
        total_dist += dist
        qv = ax1.quiver(
            xs[:-1],
            ys[:-1],
            xs[1:] - xs[:-1],
            ys[1:] - ys[:-1],
            scale_units='xy',
            angles='xy',
            scale=1,
            color=color,
            label='R{}, N({}), C {} / {}, D {:.2f}'.format(
                veh_number,
                len(r),
                int(total_route_demand) if round_demand else total_route_demand,
                int(capacity) if round_demand else capacity,
                dist
            )
        )

        qvs.append(qv)

    ax1.set_title('Sampling1280,{} routes, total distance {:.2f}'.format(len(routes), total_dist), family='Times New Roman',size=20)

    ax1.legend(handles=qvs)
    plt.legend(loc=1)
    pc_cap = PatchCollection(cap_rects, facecolor='whitesmoke', alpha=1.0, edgecolor='lightgray')
    pc_used = PatchCollection(used_rects, facecolor='lightgray', alpha=1.0, edgecolor='lightgray')
    pc_dem = PatchCollection(dem_rects, facecolor='black', alpha=1.0, edgecolor='black')

    if visualize_demands:
        ax1.add_collection(pc_cap)
        ax1.add_collection(pc_used)
        ax1.add_collection(pc_dem)
    plt.show()
    #plt.savefig("./temp{}.png".format(54), dpi=600, bbox_inches='tight')

In [110]:
import numpy as np
import pandas as pd

# Создаем DataFrame из координат
df = pd.DataFrame({
    'x': [30.22222, 30.21472, 30.22505, 30.21259, 30.22157, 30.21472, 30.20979, 30.21545, 30.21330, 30.22323, 30.20241],
    'y': [59.93778, 59.93887, 59.93950, 59.94099, 59.94315, 59.94303, 59.94417, 59.94486, 59.94567, 59.94663, 59.94515]
})

# Выводим DataFrame
print("DataFrame:")
print(df)

# Создаем массив из координат
points = df.to_numpy()

# Выводим массив точек
print("Массив точек:")
print(points)

DataFrame:
           x         y
0   30.22222  59.93778
1   30.21472  59.93887
2   30.22505  59.93950
3   30.21259  59.94099
4   30.22157  59.94315
5   30.21472  59.94303
6   30.20979  59.94417
7   30.21545  59.94486
8   30.21330  59.94567
9   30.22323  59.94663
10  30.20241  59.94515
Массив точек:
[[30.22222 59.93778]
 [30.21472 59.93887]
 [30.22505 59.9395 ]
 [30.21259 59.94099]
 [30.22157 59.94315]
 [30.21472 59.94303]
 [30.20979 59.94417]
 [30.21545 59.94486]
 [30.2133  59.94567]
 [30.22323 59.94663]
 [30.20241 59.94515]]


In [107]:
points

array([[30.22222, 59.93778],
       [30.21472, 59.93887],
       [30.22505, 59.9395 ],
       [30.21259, 59.94099],
       [30.22157, 59.94315]])

In [ ]:
datas = []
n_nodes = 11
# node_ = np.loadtxt('VRP/test_data/vrp20_test_data.csv', dtype=float, delimiter=',')
# demand_=np.loadtxt('VRP/test_data/vrp20_demand.csv', dtype=float, delimiter=',')
# demand_=np.loadtxt('VRP/test_data/demand_10.csv', dtype=float, delimiter=';')
# capcity_=np.loadtxt('VRP/test_data/vrp20_capcity.csv', dtype=float, delimiter=',')

# node_ = node_[:11]
node_ = points
demand_ = np.array([
    [0, 147, 229, 35, 528, 743, 331, 445, 456, 457, 399],
    [0, 161, 250, 38, 575, 811, 361, 485, 498, 499, 435],
    [0, 201, 313, 47, 719, 1013, 451, 606, 622, 623, 544],
    [0, 281, 437, 66, 1007, 1418, 631, 848, 871, 873, 761],
    [0, 267, 417, 63, 959, 1350, 601, 808, 829, 831, 725]
    ])
# demand_= demand_[:1]
# capcity_ =capcity_[:1]

capcity_ = np.array([2000.0]) #настройка вместимости транспорта
# node_,demand_=node_.reshape(-1,n_nodes,2),demand_.reshape(-1,n_nodes)
node_ = node_.reshape(-1, n_nodes, 2)
data_size = node_.shape[0]

# x=np.random.randint(1,data_size)
x=0 # выбираем набор данных
d = 4 # вбыираем нагрузку(день) из demand
# # Calculate the distance matrix
# edges = np.zeros((n_nodes, n_nodes, 1))
# def c_dist(x1, x2):
#     return ((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2) ** 0.5
# for i, (x1, y1) in enumerate(node_[x]):
#     for j, (x2, y2) in enumerate(node_[x]):
#         d = c_dist((x1, y1), (x2, y2))
#         edges[i][j][0] = d
# edges_ = edges.reshape(-1, 1)

np_matrix = np.asarray(time_matrix)
np_matrix = np_matrix.reshape(-1, 1)


edges_index = []
for i in range(n_nodes):
    for j in range(n_nodes):
        edges_index.append([i, j])
edges_index = torch.LongTensor(edges_index)
edges_index = edges_index.transpose(dim0=0, dim1=1)

datas = []
data = Data(x=torch.from_numpy(node_[x]).float(), edge_index=edges_index, 
            edge_attr=torch.from_numpy(np_matrix).float(),
            demand=torch.tensor(demand_[d]).unsqueeze(-1).float(),
            capcity=torch.tensor(capcity_[x]).unsqueeze(-1).float())
datas.append(data)

data_loder = DataLoader(datas, batch_size=1)


n_node = 11

agent = Model(3, 128, 1, 16, conv_laysers=4).to(device)
agent.to(device)
folder = 'trained'
filepath = os.path.join(folder, '%s' % n_nodes)
if os.path.exists(filepath):
        path1 = os.path.join(filepath, 'actor.pt')
        agent.load_state_dict(torch.load(path1, device))
datas_ = []
batch_size1 = 128  # sampling batch_size
for y in range(1280):
    data = Data(x=torch.from_numpy(node_[x]).float(), 
                edge_index=edges_index,
                edge_attr=torch.from_numpy(np_matrix).float(),
                demand=torch.tensor(demand_[d]).unsqueeze(-1).float(),
                capcity=torch.tensor(capcity_[x]).unsqueeze(-1).float())
    datas_.append(data)
dl = DataLoader(datas_, batch_size=batch_size1)

min_tour=[]
min_cost=100
T=1.2#Temperature hyperparameters
for batch in dl:
    with torch.no_grad():
        batch.to(device)
        tour1, _ = agent(batch, n_nodes * 2,False, T)
        cost = reward1(batch.x, tour1.detach(), n_nodes)
        id = np.array(cost.cpu()).argmin()
        m_cost=np.array(cost.cpu()).min()
        tour1=tour1.reshape(batch_size1,-1)
        if m_cost<min_cost:
            min_cost=m_cost
            min_tour=tour1[id]

tour=min_tour.unsqueeze(-2)

print('Problem:VRP''%s' % n_node,'/ Average distance:', min_cost)

for i, (data, tour) in enumerate(zip(data_loder, tour)):
    fig, ax = plt.subplots(figsize=(10, 10))
    plot_vehicle_routes(data, tour, ax, visualize_demands=False, demand_scale=50, round_demand=True)

In [102]:
def reward2(static, tour_indices,n_nodes):

    static = static.reshape(-1,n_nodes,2)

    static = static.transpose(2,1)

    idx = tour_indices.unsqueeze(1).expand(-1,static.size(1),-1)

    tour = torch.gather(static, 2, idx).permute(0, 2, 1)
    #print(tour.shape,tour[0])
    #print(idx.shape,idx[0])
    # Make a full tour by returning to the start
    start = static.data[:, :, 0].unsqueeze(1)
    y = torch.cat((start, tour,start), dim=1)
    tour_len = torch.sqrt(torch.sum(torch.pow(y[:, :-1] - y[:, 1:], 2), dim=2))
    return tour_len.sum(1).detach()





In [124]:
min_tour=[]
min_cost=100
T=1.2#Temperature hyperparameters
for batch in dl:
    with torch.no_grad():
        batch.to(device)
        tour1, _ = agent(batch, n_nodes * 2,False, T)
        cost = reward2(batch.x, tour1.detach(), n_nodes)
        id = np.array(cost.cpu()).argmin()
        m_cost=np.array(cost.cpu()).min()
        tour1=tour1.reshape(batch_size1,-1)
        if m_cost<min_cost:
            min_cost=m_cost
            min_tour=tour1[id]

tour=min_tour.unsqueeze(-2)

print('Problem:VRP''%s' % n_node,'/ Average distance:', min_cost)
for i, (data, tour) in enumerate(zip(data_loder, tour)):
    routes = [r[r != 0] for r in np.split(tour.cpu().numpy(), np.where(tour.cpu().numpy() == 0)[0]) if (r != 0).any()]
    print(routes)

Problem:VRP11 / Average distance: 0.1087936
[array([7, 8]), array([4, 9]), array([2, 5]), array([ 1, 10,  6,  3])]
